In [ ]:


class InventoryItem:
    def __init__(self, name, stock, price):
        self.name = name
        self.stock = stock
        self.price = price

    def sell(self, qty):
        if qty <= self.stock:
            self.stock -= qty
        else:
            raise ValueError(f"Not enough stock to sell {qty} units of {self.name}")

    def restock(self, qty):
        self.stock += qty

    def stock_value(self):
        return self.stock * self.price

    def needs_reorder(self, reorder_point):
        return self.stock <= reorder_point


class Warehouse:
    def __init__(self, name):
        self.name = name
        self.items = {}  # dict[str, InventoryItem]

    def add_item(self, item: InventoryItem):
        self.items[item.name] = item

    def transfer_to(self, other_warehouse, product_name, qty):
        if product_name not in self.items:
            raise KeyError(f"{product_name} not found in {self.name}")
        
        item = self.items[product_name]
        item.sell(qty)  # reduce stock here

        if product_name in other_warehouse.items:
            other_warehouse.items[product_name].restock(qty)
        else:
            # Create a new item in the other warehouse with same price
            new_item = InventoryItem(product_name, qty, item.price)
            other_warehouse.add_item(new_item)

    def low_stock(self, reorder_point):
        return [name for name, item in self.items.items() if item.needs_reorder(reorder_point)]

    def total_value(self):
        return sum(item.stock_value() for item in self.items.values())

In [ ]:
import statistics

def mape(actual, forecast):
    errors = [abs(a - f) / a for a, f in zip(actual, forecast)]
    return sum(errors) / len(errors) * 100



def wmape(actual, forecast):
    errors = [abs(a - f) / a for a, f in zip(actual, forecast)]
    return sum(errors) * 100
    

def cov(actual, forecast):
    stddev = [statistics.stdev(actual)
    mean = [statistics.mean(actual)

    return stddev/mean *100


products = [
  {"name":"Laptop", "actual":[100,120,130], "fcst":[90,110,125]},
  {"name":"Mobile", "actual":[200,210,190], "fcst":[195,205,180]},
  {"name":"Tablet", "actual":[50,60,55],   "fcst":[45,65,50]}
]

for product in products:
    name = product["name"]
    actual = product["actual"]
    fcst = product["fcst"]
    print(f"{name} --> MAPE: {mape(actual, fcst)} WMAPE: {wmape(actual, fcst)} COV: {cov(actual)}")



    

In [ ]:
import pandas as pd

data1 = pd.read_csv("inventorylist.csv")
print(data1)

data2 = pd.read_csv("supplier_prices.csv")
print(data2)


data = pd.merge(data, data2, on = 'Product', how = 'inner')

data["stockvalue"] = data["Stock"] * data["UnitPrice"]


data["lowstock"] = data["Stock"] < data["ReorderPoint"]
data["lowstock"] = data["lowstock"].apply(lambda x: "Yes" if x else "No")


totalstock = data.groupby('Category')['Stock'].sum()
totalStockValue = data.groupby('Category')['stockvalue'].sum()
lowstock = data.groupby('Category')['lowstock'].count()

data.to_csv("scm_pandas_merge_lowstock.csv")

In [11]:
import numpy as np


week = np.array([100,120,110,130,125,115,140,150,135,145,160,155])
lead_time_weeks = 3
safety_stock = 200


mean = np.mean(week)
stddev = np.std(week)
cov = stddev/mean *100

minweek = np.argmin(week)
maxweek = np.argmax(week)

reorder_qty = week*lead_time_weeks + safety_stock

print(f"mean --> {mean}")
print(f"stddev --> {stddev}")
print(f"cov --> {cov}")
print(f"minweek--> {minweek}")
print(f"maxweek --> {maxweek}")
print(f"reorder_qty --> {reorder_qty}")



mean --> 132.08333333333334
stddev --> 17.965050947758417
cov --> 13.60130040208839
minweek--> 0
maxweek --> 10
reorder_qty --> [500 560 530 590 575 545 620 650 605 635 680 665]


In [ ]:


import requests
import pandas as pd

# Step 1: Get data from API
response = requests.get("https://fakestoreapi.com/products")
data = response.json()

# Step 2: Convert to DataFrame
df = pd.DataFrame(data)

# Step 3: Rename columns for clarity
df["Product"] = df["title"]
df["SupplierPrice"] = df["price"]
df["Rating"] = df["rating"].apply(lambda x: x["rate"])

# Step 4: Top 5 by price
top5price = df.sort_values(by='SupplierPrice', ascending=False).head(5)

# Step 5: Group by category
avgprice = df.groupby('category')['SupplierPrice'].mean()
count = df.groupby('category')['Product'].count()

# Step 6: Print results
print("Top 5 Products by Price:")
print(top5price[["Product", "SupplierPrice", "category", "Rating"]])

print("\nCategory Summary:")
print("Average Price:\n", avgprice)
print("Product Count:\n", count)

# Step 7: Save to CSV
df.to_csv("supplier_catalog_api.csv", index=False)